# Accessing generated particles to calculate production fractions

In this example, we access the container of generated particles (`genParticles`) and count the number of different $B$-hadron species produced in the $Z^0 \to b\bar{b}$ hadronisation.

In [1]:
import uproot4 as uproot
import awkward1 as ak
import sys,os
import json
import numpy as np
from fcc_python_tools.locations import loc
from fcc_python_tools.particle_properties import particles

Let's load the `ROOT` file with `uproot` and access the `events` tree:

In [2]:
file = uproot.open(f"{loc.DATA}/FCCDelphesOutput.root")
tree = file['events']

Next we load the `genParticles` container into an `awkward array`, using the wildcard `*` to load all branches.

In [3]:
gen_container = "genParticles.core"
gen = tree.arrays(filter_name=f"{gen_container}*",how="zip")

There is a helper dictionary in the `fcc_python_tools/particle_properties.py` script called `particles`, which contains the masses (`m`) and PDG IDs (`pdgId`) of several particles. We use it here to filter out specific $B$-hadrons to count how many there are. You can add more particles to the dict if you need to.

Here we loop over different $B$-hadron types, using their `pdgId` values to filter and then count how many there are:

In [4]:
b_types = {"Bd": "$B^0$", 
           "Bu": "$B^\pm$", 
           "Bs": "$B_s^0$", 
           "Bc": "$B_c^\pm$",
           "Lb": "$\\Lambda_b^0$"
          }
n = {}
for b in b_types:
    b_cut = abs(gen[gen_container,"pdgId"]) == particles[b]["pdgId"]
    n[b] = ak.sum(ak.flatten(b_cut))
print(n)

{'Bd': 8615, 'Bu': 8620, 'Bs': 1963, 'Bc': 8, 'Lb': 698}


Here we have used `ak.flatten()` to turn the jagged array (more than one particle per event) into a flat array. We then use the `ak.sum()` function to count the total number of each $B$-hadron species across all events.

We want to know what fraction of the $b$-quarks hadronise to each of the $B$-hadrons. To calculate this, we need the total number of $b$-quarks produced. This one is easy: since we are dealing with $Z^0 \to b\bar{b}$, every event contains 2 $b$-quarks. So the total number of $b$-quarks is just twice the total number of events in our `ROOT` file. 

In [5]:
n_bbbar = 2*tree.num_entries

Let's calculate the production fraction for each $B$-hadron:

In [6]:
prod_frac = {}
for b in b_types:
    prod_frac[b] = 100*(n[b] / n_bbbar)
print(prod_frac)

{'Bd': 43.075, 'Bu': 43.1, 'Bs': 9.815, 'Bc': 0.04, 'Lb': 3.49}


## Persisting the results

We can store these values in a dictionary and write it to a `.json` file. `JSON` format is great whenever you need to re-load some analysis numbers you have calculated within another script - it is an important form of analysis persistence.

In [7]:
with open(f'{loc.JSON}/b_hadron_prod_fracs.json', 'w') as f:
    json.dump(prod_frac, f, sort_keys=True, indent=4)

Let's finish off by making a LaTeX summary table of our production fractions. We can write this to a `.tex` file which we can then put into some documentation or Beamer slides.

In [8]:
f_tex = open(f'{loc.TABLES}/b_hadron_prod_fracs.tex','w')
f_tex.write('\\begin{table}[!h] \n')
f_tex.write('\\centering \n')
f_tex.write('\\begin{tabular}{l c} \n')
f_tex.write('$B$-hadron & Production fraction (\%) \\\\ \\hline \n')
for b in b_types:
    f_tex.write(f'{b_types[b]} & {prod_frac[b]:.2f} \\\\ \n')
f_tex.write('\\end{tabular} \n')
f_tex.write('\\caption{$B$-hadron production fractions in $Z^0 \\to b\\bar{b}$ simulation. \\label{tab:b_hadron_prod_fracs}} \n')
f_tex.write('\\end{table}')
f_tex.close()

Note the use of double backslash (`\\`) here, which is required in order for the `write()` command to interpret the backslash properly and print it. The use of `\n` is different, as this is the command to print on a new line next time `write()` is called. 